In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split


nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukakrickovic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Ucitavanje dataseta

In [2]:
df = pd.read_csv('../datasets/fake-news/balansed_fake_news_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49550 entries, 0 to 49549
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    49550 non-null  object
 1   text     49548 non-null  object
 2   subject  49550 non-null  object
 3   date     34190 non-null  object
 4   fake     49550 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.6+ MB


#### Ciscenje teksta

In [3]:
df = df.reset_index(drop=True)

def clean_text(text: str) -> str:
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords)
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['text'].dropna()
df = df.loc[df.text.apply(type) != float]
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

/var/folders/ny/km1m35px577gl9cs19xd2scm0000gn/T/ipykernel_1611/4199856113.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')


## GloVe

In [4]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [5]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(df.text)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(df.text)

In [6]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [7]:
import zipfile
import urllib.request

pth = 'glove.6B.zip'
output_txt = 'glove.6B'
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip',pth)
with zipfile.ZipFile(f'./{pth}', 'r') as zip_ref:
    zip_ref.extractall('glove.6B')

In [8]:
# load the whole embedding into memory
embeddings_index = {}

with open('./glove.6B/glove.6b.100d.txt', encoding="utf8") as f:
	for line in f:
		values = line.split()
		word = values[0]
		coefs = asarray(values[1:], dtype='float32')
		embeddings_index[word] = coefs

print(f'Loaded {len(embeddings_index)} word vectors.')

Loaded 400000 word vectors.


In [9]:
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [10]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the modele
print(model.summary())

2022-04-25 16:26:59.743388: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            25007700  
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 1)                 401       
                                                                 
Total params: 25,008,101
Trainable params: 401
Non-trainable params: 25,007,700
_________________________________________________________________
None


In [11]:
# fit the model
model.fit(padded_docs, df.fake, epochs=50, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, df.fake, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 82.970047
